In [1]:
from espn_api.basketball import League, Matchup, Team
from activity.myUtil import get_opponent_team, get_my_team
from espn_api.basketball.week import Week
from espn_api.basketball.roster_week_predictor import RosterWeekPredictor

from IPython.display import HTML, display
import tabulate

In [2]:
espn_s2 = "AECc%2BKMx5L8l9Rai5PHG%2FT%2Fjtw%2BR%2FZozX391uxpta1J6tD61%2F4NV88%2BVPlCSe0Wjcg0J9jJ0oTJ%2BMkehJLT8MSDfgiubWB9F1n3H%2BnTHWc9OTkzM2ceQFGPiA%2B9AF4yQzt0I3%2FdCgk%2BdQLbgybhS6y1yRS9NIDyfXMSgcqFqRbn5F7zBTzwu9A%2BD2Pi6g6KMR29PbmNbtxSrCJkfbF%2BaZtrpqDbHBZhLyfqOAhQIDx0q1iD0u1DkhxVNoRnxx%2FmEvS%2FHEJZWmfvZM%2F0IAHxPrfVx"
swid="{3C4A75B6-F84B-48AE-8A75-B6F84B48AE47}"

In [3]:
league = League(league_id=30695, year=2021, espn_s2=espn_s2, swid=swid)

In [4]:
table_output = [("Team Name", "Team Id")]
[table_output.append((team.team_name, team.team_id)) for team in league.teams]
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,Team Id
CZ ButtKicker,1
SEA MoNeYPro,2
NY Zhanos Diesel,4
Shanghai SHARKS,5
Team Tao,7
Team Beyond,9
PHI Lower Merion 33,11
Badwomen ``,14
Dragon City Tiedan,15
Tim Liang,16


Put your **team id** into the variable below:

In [10]:
MY_TEAM_ID = 2

In [11]:
schedule = league.get_team_data(MY_TEAM_ID).schedule
table_output = [("Week Id", "Opponent Team Name")]
[table_output.append((week_id + 1, get_opponent_team(matchup, MY_TEAM_ID).team_name)) for week_id, matchup in enumerate(schedule)]
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Week Id,Opponent Team Name
1,Tim Liang
2,Dragon City Tiedan
3,Team Beyond
4,PHI Lower Merion 33
5,Team Tao
6,CZ ButtKicker
7,NY Zhanos Diesel
8,Badwomen ``
9,Team Tao
10,Tim Liang


In [7]:
WEEK_ID = 3

In [8]:
currWeek = Week(league, WEEK_ID)

In [9]:
predicted_points_team_name_map = {}
number_of_games_team_name_map = {}
for team in league.teams:
    predictor = RosterWeekPredictor(team.roster, currWeek)
    predicted_points_team_name_map[team.team_name] = predictor.predict()
    number_of_games_team_name_map[team.team_name] = predictor.get_total_number_of_games()

In [10]:
def get_tuple_average(tup):
    return (tup[0] + tup[1]) / 2

In [11]:
team_scores = {}
for team in league.teams:
    predicted_points = predicted_points_team_name_map[team.team_name]
    team_scores[team.team_name] = predicted_points[0], predicted_points[1], get_tuple_average(predicted_points)

In [12]:
table_output = []
for team_name, scores in team_scores.items():
    lo, hi, avg = scores
    table_output.append((team_name, number_of_games_team_name_map[team_name], lo, hi, avg))
table_output.sort(reverse=True, key=lambda x: x[-1])

In [13]:
table_output.insert(0, ("Team Name", "# of games", "Low", "High", "Avg"))
display(HTML(tabulate.tabulate(table_output, tablefmt='html')))

Team Name,# of games,Low,High,Avg
Shanghai SHARKS,41,1383.1045454545454,1569.4229437229437,1476.2637445887444
Team Tao,42,1343.5195887445889,1594.5616883116882,1469.0406385281385
SEA MoNeYPro,42,1288.8454545454547,1524.9333333333334,1406.889393939394
Tim Liang,42,1231.3232323232323,1457.794155844156,1344.5586940836943
NY Zhanos Diesel,37,1152.5963203463205,1314.8051948051948,1233.7007575757575
Team Beyond,40,1122.4525974025973,1319.7285714285715,1221.0905844155845
Badwomen ``,35,1129.2818181818182,1277.0343073593074,1203.1580627705628
Dragon City Tiedan,34,925.9707792207793,1144.780303030303,1035.375541125541
CZ ButtKicker,36,915.1944444444445,1144.1704545454547,1029.6824494949497
PHI Lower Merion 33,32,913.8787878787878,1100.3280303030303,1007.103409090909


In [14]:
match_up_points = []
match_up_points.append(["Home Team", "Estimate Points", "# of Games", "Away Team", "Estimate Points", "# of Games", "+/-"])
for matchup in league.scoreboard():
    home_team_average = team_scores[matchup.home_team.team_name][-1]
    away_team_average = team_scores[matchup.away_team.team_name][-1]
    match_up_points.append([matchup.home_team.team_name, home_team_average, number_of_games_team_name_map[matchup.home_team.team_name],
                            matchup.away_team.team_name, away_team_average, number_of_games_team_name_map[matchup.away_team.team_name],
                           home_team_average - away_team_average])

In [15]:
display(HTML(tabulate.tabulate(match_up_points, tablefmt='html')))

Home Team,Estimate Points,# of Games,Away Team,Estimate Points,# of Games,+/-
Team Beyond,1221.0905844155845,40,Badwomen ``,1203.1580627705628,35,17.93252164502178
Tim Liang,1344.5586940836943,42,Team Tao,1469.0406385281385,42,-124.48194444444425
NY Zhanos Diesel,1233.7007575757575,37,Shanghai SHARKS,1476.2637445887444,41,-242.56298701298692
PHI Lower Merion 33,1007.103409090909,32,SEA MoNeYPro,1406.889393939394,42,-399.785984848485
CZ ButtKicker,1029.6824494949497,36,Dragon City Tiedan,1035.375541125541,34,-5.693091630591425


In [16]:
league.get_team_data(MY_TEAM_ID).roster[0].eligibleSlots

['PG', 'G', 'G/F', 'UT', 'BE', 'IR']

In [7]:
from espn_api.basketball.line_up_editer import LineUpEditor

editor = LineUpEditor(league, MY_TEAM_ID)
# editor.bench_all_players(MY_TEAM_ID, 14)

In [8]:
editor.get_optimized_line_up(league.get_team_data(MY_TEAM_ID).roster)

[(4277905, 0),
 (3149673, 3),
 (6477, 4),
 (2527963, 1),
 (4065648, 2),
 (4251, 11),
 (4432816, 11),
 (3064290, 11),
 (4277847, 11),
 (3064440, 11)]

In [9]:
editor.fill_line_up(25)

Failed to bench player  Player(Jayson Tatum)
Failed to change line up for player  4065648
